## Loading Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v2 import MobileNetV2
from keras import Sequential
from keras.layers import Flatten, Dense

## Data Loading and Augmentation

In [2]:
train_dir = r'Face Mask Dataset/Train'
test_dir = r'Face Mask Dataset/Test'
validation_dir = r'Face Mask Dataset/Validation'

In [3]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(128, 128),  # All images will be resized to 150x150
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical')

Found 10023 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


## Importing Model

In [4]:
mobilenet_v2 = MobileNetV2(weights='imagenet',include_top=False,input_shape=(128,128,3))

for layer in mobilenet_v2.layers:
    layer.trainable = False

In [5]:
model = Sequential()
model.add(mobilenet_v2)
model.add(Flatten())
model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_128 (Functi (None, 4, 4, 1280)        2257984   
_________________________________________________________________
flatten (Flatten)            (None, 20480)             0         
_________________________________________________________________
dense (Dense)                (None, 2)                 40962     
Total params: 2,298,946
Trainable params: 40,962
Non-trainable params: 2,257,984
_________________________________________________________________


In [6]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics ="accuracy")

## Training

In [ ]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=len(train_generator)//32,
                              epochs=20,validation_data=validation_generator,
                              validation_steps=len(validation_generator)//32)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
9/9 [==============================] - 11s 1s/step - loss: 0.3146 - accuracy: 0.9062
Epoch 2/20
9/9 [==============================] - 10s 1s/step - loss: 0.3442 - accuracy: 0.9306
Epoch 3/20
9/9 [==============================] - 9s 1s/step - loss: 0.1627 - accuracy: 0.9618
Epoch 4/20
9/9 [==============================] - 9s 1s/step - loss: 0.2909 - accuracy: 0.9479
Epoch 5/20
9/9 [==============================] - 7s 832ms/step - loss: 0.1556 - accuracy: 0.9757
Epoch 6/20
9/9 [==============================] - 8s 846ms/step - loss: 0.2314 - accuracy: 0.9583
Epoch 7/20
9/9 [==============================] - 7s 799ms/step - loss: 0.1469 - accuracy: 0.9653
Epoch 8/20
9/9 [==============================] - 6s 721ms/step - loss: 0.1047 - accuracy: 0.9792
Epoch 9/20
9/9 [==============================] - 6s 664ms/step - loss: 0.2099 - accuracy: 0.9688
Epoch 10/20
9/9 [==============================] - 6

In [8]:
model.save('masknet.h5')

## Testing Model

In [ ]:
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = train_datagen.flow_from_directory(directory=test_dir,target_size=(128,128),class_mode='categorical',batch_size=32)

In [ ]:
model.evaluate_generator(test_generator)

In [ ]:
import cv2 
import matplotlib.pyplot as plt
import numpy as np
sample_mask_img = cv2.imread(test_dir+r'\Withmask\190.png')
sample_mask_img = cv2.resize(sample_mask_img,(128,128))
plt.figure(figsize=(1,1))
x=cv2.cvtColor(sample_mask_img, cv2.COLOR_BGR2RGB)
plt.imshow(x)
sample_mask_img = np.reshape(sample_mask_img,[1,128,128,3])
sample_mask_img = sample_mask_img/255.0

In [ ]:
model.predict(sample_mask_img)